In [6]:
from dotenv import load_dotenv
from openai import OpenAI
import os


In [12]:
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"


openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

models = ["gpt-5", "claude-sonnet-4-5-20250929" ]

openai= OpenAI(api_key=openai_api_key)
anthropic = OpenAI(base_url=anthropic_url,api_key=anthropic_api_key)
clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic}


In [29]:
system_prompt = """
You will be given a python code convert to an Pytorch code both will run in CPUs only. I want no additional text other then the code explain anthing you want in the comments
"""

def user_prompt_for(python):
  return f"""
  Here is my python code {python} to be converted give the resultant pytorch code with code to see how much time it takes to run
  """

In [30]:
def messages_for(python):
  return [{'role':'user','content':user_prompt_for(python)},{'role':'system','content':system_prompt}]


In [36]:
def port(model,python):
  client = clients[model]
  response = client.chat.completions.create(model=model,messages=messages_for(python))
  reply = response.choices[0].message.content
  reply = reply.replace('```python','').replace('```','')
  return reply
  

In [40]:
import io
import contextlib
def run_pythons(code):
  buffer = io.StringIO()
  with contextlib.redirect_stdout(buffer):
        exec(code)
  return buffer.getvalue()



In [41]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
import gradio as gr

with gr.Blocks() as ui:
  with gr.Row():
    python = gr.Code(label = "Python code: ",lines=28,value=pi)
    pytorch = gr.Code(label="Pytorch code: ",lines=28)
  with gr.Row():
    model = gr.Dropdown(models,label="Select model",value=models[0])
    convert = gr.Button("Convert code")
  with gr.Row():
    run_python = gr.Button("Run Python")
    run_pytorch = gr.Button("Run Pytorch")
  with gr.Row():
    python_out = gr.TextArea(label="Python",lines=8)
    py_out = gr.TextArea(label="Pytorch",lines=8)
  run_python.click(run_pythons,inputs=[python],outputs=[python_out])
  run_pytorch.click(run_pythons,inputs=[python],outputs=[pytorch_out])
  convert.click(port,inputs=[model,python],outputs=[pytorch])
ui.launch(inbrowser=True)

NameError: name 'out' is not defined